# Prepare Dataset

In [1]:
import os
#hfhome_dir="/data/cmpe249-fa23/Huggingfacecache"
hfhome_dir=os.path.join('D:',os.sep, 'Cache','huggingface')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
#os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1

In [2]:
from datasets import load_dataset, load_metric
dataset_split = load_dataset(
        "librispeech_asr", #'librispeech_asr'
        'clean',
        split='train.100',
    )

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_split

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 28539
})

In [4]:
dataset_iter = iter(dataset_split)
sample = next(dataset_iter)

In [5]:
sample

{'file': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\a77e60630727409949078da012aba02689008a522ed19f5f43300c735482710c\\374-180298-0000.flac',
 'audio': {'path': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\a77e60630727409949078da012aba02689008a522ed19f5f43300c735482710c\\374-180298-0000.flac',
  'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
         -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED',
 'speaker_id': 374,
 'chapter_id': 180298,
 'id': '374-180298-0000'}

In [6]:
audioarray=sample["audio"]["array"]
print(len(audioarray))

232480


In [11]:
sample["audio"]["sampling_rate"]

16000

In [7]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch

In [8]:
# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")


config.json: 100%|██████████| 1.60k/1.60k [00:00<00:00, 1.59MB/s]
model.safetensors: 100%|██████████| 378M/378M [00:03<00:00, 116MB/s]  
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametriz

In [12]:
#forward sample through model to get greedily predicted transcription ids
processedsample=feature_extractor(sample["audio"]["array"],sampling_rate=sample["audio"]["sampling_rate"], return_tensors="pt")

In [13]:
processedsample

{'input_values': tensor([[ 0.0117,  0.0122,  0.0122,  ..., -0.0041, -0.0026, -0.0001]])}

In [14]:
input_values = processedsample.input_values
logits = model(input_values).logits[0]

In [16]:
logits.shape

torch.Size([726, 32])

In [17]:
pred_ids = torch.argmax(logits, axis=-1)
pred_ids

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 19, 19, 11, 11,  0,  0,  0,
         0,  7,  0, 23,  0,  0,  0,  6,  0,  0,  5,  5, 13, 13,  0,  4,  4,  4,
         0,  0,  0,  0,  0, 12,  0,  0,  0, 10, 10,  0, 28,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  6,  6,  0,  0,  5,  0,  0,  0,  0,  0,  0,
         5,  0,  9,  9,  0,  0,  0,  4,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  4,  4,  4, 17, 17,  0,  0,
        10, 21, 11, 11,  6,  6,  0,  4,  4,  0, 11,  0,  7, 25, 25,  5,  4,  4,
         0,  0,  6,  0,  0,  0,  8,  8, 15, 15,  0, 14, 14,  4,  4,  4, 22, 22,
         0,  8, 16, 16,  0,  4,  4,  4, 

In [18]:
pred_ids.shape

torch.Size([726])

In [19]:
# retrieve word stamps (analogous commands for `output_char_offsets`)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)

In [21]:
# compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

In [23]:
word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in outputs.word_offsets
]

In [24]:
word_offsets[:3]

[{'word': 'CHAPTER', 'start_time': 0.22, 'end_time': 0.64},
 {'word': 'SIXTEEN', 'start_time': 0.82, 'end_time': 1.52},
 {'word': 'I', 'start_time': 3.42, 'end_time': 3.44}]

In [30]:
dataset = dataset_split.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

In [31]:
from transformers import AutoProcessor, Wav2Vec2ForCTC
# transcribe speech
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(pred_ids)
transcription[0]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

'<pad>'

In [32]:
inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

25.74

# Fine-tuning Wav2Vec2 for English ASR

In [34]:
from datasets import DatasetDict
raw_datasets = DatasetDict()
raw_datasets["train"] = dataset_split
validation_split_percentage = 1
num_validation_samples = raw_datasets["train"].num_rows * validation_split_percentage // 100

valkeyname="test"
raw_datasets[valkeyname] = raw_datasets["train"].select(range(num_validation_samples))
raw_datasets["train"] = raw_datasets["train"].select(range(num_validation_samples, raw_datasets["train"].num_rows))

In [35]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 28254
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 285
    })
})

In [36]:
raw_datasets = raw_datasets.remove_columns(["speaker_id", "chapter_id", "id"])

In [37]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 28254
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 285
    })
})

In [38]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [39]:
show_random_elements(raw_datasets["train"].remove_columns(["file", 'audio']), num_examples=20)

,text
0,MIGHT HAVE SEEN THAT THE CHIN WAS VERY POINTED AND PRONOUNCED THAT THE BIG EYES WERE FULL OF SPIRIT AND VIVACITY THAT THE MOUTH WAS SWEET LIPPED AND EXPRESSIVE THAT THE FOREHEAD WAS BROAD AND FULL IN SHORT OUR DISCERNING EXTRAORDINARY OBSERVER MIGHT HAVE CONCLUDED
1,THE OTHER RETIRING BACKWARD AND BOWING WITH HIS HANDS ON HIS KNEES REPLIED INDEED INDEED I AM FILLED WITH ADMIRATION AT THE GOODNESS OF YOUR HEART WHEN I HEAR YOU SPEAK THUS I FEEL MORE THAN EVER HOW GREAT IS THE LOVE I BEAR YOU
2,SO ADAM GOT ON HIS HORSE AGAIN AND RODE TO THE TOWN PUTTING UP AT THE OLD INN AND TAKING A HASTY DINNER THERE IN THE COMPANY OF THE TOO CHATTY LANDLORD FROM WHOSE FRIENDLY QUESTIONS AND REMINISCENCES
3,BUT THE KNOWLEDGE ONLY ANGERED HER FURTHER
4,EXCEPT JERMYN HILLIARD JUNIOR AND HE FOLLOWED ME RIGHT INTO THE PARLOR CAR AND SAT DOWN IN THE CHAIR EXACTLY OPPOSITE PATTY THEY CRIED IN SHOCKED CHORUS YOU SURELY DIDN'T SPEAK TO HIM
5,I THEREFORE BOUND IT CLOSELY TO HIS SIDE AND ASKED HIM IF IN HIS EXHAUSTED AND TREMBLING CONDITION HE WAS STILL ABLE TO WALK YES HE BRAVELY REPLIED SO WITH A STEADYING ARM AROUND HIM AND MANY STOPS FOR REST
6,INTO MY OWN BERKSHIRE HILLS THAT SIT ALL SADLY GUARDING THE GATES OF MASSACHUSETTS UP THE STAIRS I RAN TO THE WAN MOTHER AND WHIMPERING BABE TO THE SANCTUARY ON WHOSE ALTAR A LIFE AT MY BIDDING
7,AND IN A WORD PRESSED ME TO BUY BETTER
8,BUT HER MIND WAS VERY FAR FROM THE CORRESPONDENCE BEFORE HER SHE HEARD THE SOFT THUD OF THE FRONT DOOR CLOSING AND RISING SHE CROSSED THE ROOM RAPIDLY AND LOOKED DOWN THROUGH THE WINDOW TO THE STREET SHE WATCHED FISHER UNTIL HE WAS OUT OF SIGHT
9,RAPIDLY THE AIRSHIP ASCENDED AND WHEN IT WAS HIGH OVER THE TOWN OF SHOPTON TOM HEADED THE CRAFT DUE WEST LOOKING DOWN HE TRIED TO DESCRY MARY NESTOR IN HER CARRIAGE BUT THE TREES WERE IN THE WAY


In [40]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [11]:
raw_datasets = raw_datasets.map(remove_special_characters)

In [41]:
show_random_elements(raw_datasets["train"].remove_columns(["file", "audio"]))

,text
0,HE JUST HAD TO FIND OUT IF THERE WAS ANYTHING NEW OVER THERE HARDLY HAD HE REACHED IT WHEN HE HEARD A PLAINTIVE VOICE CRYING PEE WEE PEE WEE PEE WEE PETER CHUCKLED HAPPILY
1,BUT HE DOES NOT PERCEIVE THE REASONS THAT LED KEATS TO ALTER THIS IN THE VERSION HE PUBLISHED IN LEIGH HUNT'S INDICATOR TO AH WHAT CAN AIL THEE WRETCHED WIGHT AND SO ON
2,THAT THEY HAD TRAITOROUSLY ENDEAVORED TO SUBVERT THE FUNDAMENTAL LAWS AND GOVERNMENT OF THE KINGDOM TO DEPRIVE THE KING OF HIS REGAL POWER AND TO IMPOSE ON HIS SUBJECTS AN ARBITRARY AND TYRANNICAL AUTHORITY THAT THEY HAD ENDEAVORED
3,THOUGH EVEN YET THERE WAS A SECOND HURRIEDLY APPREHENSIVE GLANCE TOWARD THE DOOR BEFORE SHE SPOKE THE LOWESTOFT YES I'M SO GLAD THAT IS OF COURSE I MUST BE GLAD I'LL GET IT HER VOICE BROKE AS SHE PULLED HERSELF FROM HER CHAIR
4,I WANT YOU TO ASK HER WHAT SHE THOUGHT OF THE PARTY I GAVE ON SUNDAY LAST DID YOU HOLD YOUR TONGUE AND DO AS I TELL YOU SHE'LL BE JEALOUS AND ASK YOU WHETHER YOU WERE PRESENT OF COURSE YOU WEREN'T FOR THERE WAS NO PARTY YOU'LL BOTH EXPRESS DISCONTENT
5,AND I HOPE THAT MY READERS ARE NOW READY TO CONCLUDE WITH ME THAT THE PRETENDED SPIRITUALITY OF OUR EMOTIONS AND OF OUR ATTRIBUTES OF VALUE SO FAR FROM PROVING AN OBJECTION TO THE PHILOSOPHY OF PURE EXPERIENCE DOES WHEN RIGHTLY DISCUSSED AND ACCOUNTED FOR
6,BUT OF COURSE IF THE HYPNOTIZER LOOKS STEADILY INTO THE EYES OF HIS SUBJECT AND THE SUBJECT LOOKS INTO HIS EYES THE STEADY GAZE ON A BRIGHT OBJECT WILL PRODUCE HYPNOTISM IN ONE QUITE AS READILY AS IN THE OTHER
7,BUT WHEN SHE ASKED FOR SOMETHING THAT MEANT MORE THAN LIFE TO HER IT WAS REFUSED OF COURSE SHE HAD GONE THROUGH ALL SORTS OF HUMILIATION TO GET HIM THAT MONEY AND THIS WAS THE GRATITUDE SHE RECEIVED GRAHAM LISTENED
8,AND WE NEED NOT BE APPREHENSIVE THAT THERE WILL BE TOO MUCH STABILITY WHILE THERE IS EVEN THE OPTION OF CHANGING NOR NEED WE DESIRE TO PROHIBIT THE PEOPLE FROM CONTINUING THEIR CONFIDENCE WHERE THEY THINK IT MAY BE SAFELY PLACED AND WHERE BY CONSTANCY ON THEIR PART
9,WERE HE LOVER HUSBAND OR SON SO AMABEL BELIEVED AND SO WOULD THESE OTHERS BELIEVE ALSO WHEN ONCE RELIEVED OF THE MAGNETIC PERSONALITY OF THIS EXTRAORDINARY WITNESS


In [42]:
import transformers
from transformers import (
    AdamW,
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices

In [166]:
model_name_or_path="facebook/wav2vec2-base-960h" #"facebook/wav2vec2-large-lv60" #"facebook/wav2vec2-base-960h" #"patrickvonplaten/wav2vec2-base-v2" "facebook/wav2vec2-base"

In [167]:
#option1
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

#option2
# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [164]:
feature_extractor.sampling_rate

16000

In [165]:
feature_extractor.do_normalize

True

In [168]:
from transformers import Wav2Vec2CTCTokenizer
#option1
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

#option2
# tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [169]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [170]:
import datasets
# make sure that dataset decodes audio with correct sampling rate
raw_datasets = raw_datasets.cast_column(
    "audio", datasets.features.Audio(sampling_rate=feature_extractor.sampling_rate)
)

In [171]:
# set max & min audio length in number of samples
max_duration_in_seconds = 5 
min_duration_in_seconds = 3   
max_length = int(max_duration_in_seconds * feature_extractor.sampling_rate) #80000
min_length = int(min_duration_in_seconds * feature_extractor.sampling_rate) #48000

In [172]:
# def prepare_dataset(batch):
#     sample = batch["audio"]

#     inputs = feature_extractor(
#         sample["array"], sampling_rate=sample["sampling_rate"], max_length=max_length, truncation=True
#     )
#     batch["input_values"] = inputs.input_values[0]
#     batch["input_length"] = len(inputs.input_values[0])

#     return batch

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor(): #redirected to Wav2Vec2CTCTokenizer
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

When wrapping the processor into the as_target_processor context, however, the same method is redirected to Wav2Vec2CTCTokenizer's call method.

In [173]:
cache_file_names = None
vectorized_datasets = raw_datasets.map(
        prepare_dataset,
        num_proc=1,
        remove_columns=raw_datasets["train"].column_names,
        cache_file_names=cache_file_names,
    )

Map:   0%|          | 0/28254 [00:00<?, ? examples/s]c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 285/285 [00:02<00:00, 99.04 examples/s] 


In [174]:
#filter all sequences that are longer than 4 seconds out of the training dataset.
if max_length > 0.0:
    vectorized_datasets = vectorized_datasets.filter(
        lambda x: x < max_length,
        num_proc=1,
        input_columns=["input_length"],
    )

Filter: 100%|██████████| 285/285 [00:00<00:00, 22991.11 examples/s]


In [176]:
vectorized_datasets['train']

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 1930
})

In [177]:
vectorized_datasets['train'][0]['labels']

[24,
 16,
 6,
 4,
 22,
 8,
 16,
 4,
 14,
 10,
 14,
 9,
 27,
 6,
 4,
 20,
 8,
 13,
 21,
 5,
 6,
 4,
 11,
 5,
 13,
 4,
 7,
 16,
 9,
 6,
 4,
 17,
 7,
 16,
 14,
 4,
 10,
 9,
 6,
 5,
 13,
 23,
 8,
 12,
 5,
 14,
 4,
 18,
 10,
 6,
 11,
 4,
 7,
 15,
 17,
 8,
 12,
 6,
 4,
 17,
 5,
 9,
 7,
 19,
 10,
 9,
 21,
 4,
 7,
 13,
 19,
 11,
 9,
 5,
 12,
 12]

In [178]:
vectorized_datasets = vectorized_datasets.remove_columns("input_length")

In [179]:
vectorized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1930
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 10
    })
})

Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. E.g., a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator

In [180]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

#ref from https://github.com/huggingface/transformers/blob/main/examples/research_projects/wav2vec2/run_asr.py
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [181]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [36]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB 8.6 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.6 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 13.0 MB/s eta 0:00:00


In [182]:
wer_metric = load_metric("wer")

In [183]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [184]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    model_name_or_path,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [185]:
model.freeze_feature_extractor()

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [186]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./output/wav2vec2-finetune-asr",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [187]:
valkeyname

'test'

In [188]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets[valkeyname],
    tokenizer=processor.feature_extractor,
)

In [189]:
trainer.train()

trainer.push_to_hub()

## Evaluate

In [115]:
raw_datasets["test"]

Dataset({
    features: ['file', 'audio', 'text'],
    num_rows: 285
})

In [117]:
#facebook/wav2vec2-large-lv60
from transformers import AutoProcessor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

sampling_rate = raw_datasets["test"].features["audio"].sampling_rate

In [118]:
sampling_rate

16000

In [119]:
eval_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
eval_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [151]:
dataset[0]

{'file': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\a77e60630727409949078da012aba02689008a522ed19f5f43300c735482710c\\103-1240-0000.flac',
 'audio': {'path': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\a77e60630727409949078da012aba02689008a522ed19f5f43300c735482710c\\103-1240-0000.flac',
  'array': array([-0.00650024, -0.00552368, -0.00619507, ...,  0.0032959 ,
          0.00045776, -0.00946045]),
  'sampling_rate': 16000},
 'text': 'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK',
 'speaker_id': 103,
 'chapter_id': 1240,
 'id': '103-1240-0000'}

In [150]:
# audio file is decoded on the fly
inputs = eval_processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
inputs=inputs.to('cuda')
with torch.no_grad():
    logits = eval_model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

In [152]:
pred_str = eval_processor.batch_decode(predicted_ids)[0]
pred_str
#text = eval_processor.decode(dataset[0]["text"], group_tokens=False)

'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK'

In [153]:
dataset[0]["text"]

'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK'

In [122]:
# transcribe speech
transcription = eval_processor.batch_decode(predicted_ids)
transcription[0]


'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK'

In [154]:
inputs["labels"] = eval_processor(text=dataset[0]["text"], return_tensors="pt").input_ids

In [156]:
inputs

{'input_values': tensor([[-0.3012, -0.2587, -0.2879,  ...,  0.1243,  0.0010, -0.4297]],
       device='cuda:0'), 'labels': tensor([[19, 11,  7, 23,  6,  5, 13,  4,  8,  9,  5,  4, 17, 10, 12, 12, 16, 12,
          4, 13,  7, 19, 11,  5, 15,  4, 15, 22,  9, 14,  5,  4, 10, 12,  4, 12,
         16, 13, 23, 13, 10, 12,  5, 14,  4, 17, 10, 12, 12, 16, 12,  4, 13,  7,
         19, 11,  5, 15,  4, 15, 22,  9, 14,  5,  4, 15, 10, 25,  5, 14,  4, 29,
         16, 12,  6,  4, 18, 11,  5, 13,  5,  4,  6, 11,  5,  4,  7, 25,  8,  9,
         15,  5,  7,  4, 17,  7, 10,  9,  4, 13,  8,  7, 14,  4, 14, 10, 23, 23,
          5, 14,  4, 14,  8, 18,  9,  4, 10,  9,  6,  8,  4,  7,  4, 15, 10,  6,
          6, 15,  5,  4, 11,  8, 15, 15,  8, 18,  4, 20, 13, 10,  9, 21,  5, 14,
          4, 18, 10,  6, 11,  4,  7, 15, 14,  5, 13, 12,  4,  7,  9, 14,  4, 15,
          7, 14, 10,  5, 12,  4,  5,  7, 13, 14, 13,  8, 23, 12,  4,  7,  9, 14,
          4,  6, 13,  7, 25,  5, 13, 12,  5, 14,  4, 24, 22,  4,  7

In [155]:
# compute loss, inputs as labels and input_values
loss = eval_model(**inputs).loss
round(loss.item(), 2)

25.73

In [77]:
mymodel = Wav2Vec2ForCTC.from_pretrained(".\output\wav2vec2-finetune-asr\checkpoint-7000")

In [82]:
mymodel=mymodel.cuda()

In [127]:
eval_model=eval_model.cuda()

In [78]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True)

In [142]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = eval_model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = eval_processor.batch_decode(pred_ids)[0]
  batch["text"] = eval_processor.decode(batch["labels"], group_tokens=False)
  
  return batch

we will make use of the map(...) function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary "results".

Note: we evaluate the test data set with batch_size=1 on purpose due to this issue. Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [141]:
vectorized_datasets["test"]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10
})

In [159]:
labelresult=eval_processor.decode(vectorized_datasets["test"][0]['labels'], group_tokens=False) 

In [160]:
labelresult

'XXDXXXDXXXXXXXXXXDXXDXXXXXXDXXDXXXXXXXXX'

In [143]:
results = vectorized_datasets["test"].map(map_to_result, remove_columns=vectorized_datasets["test"].column_names)

Map: 100%|██████████| 10/10 [00:00<00:00, 10.36 examples/s]


compute the overall WER

In [144]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 10.900


In [145]:
results

Dataset({
    features: ['pred_str', 'text'],
    num_rows: 10
})

In [146]:
show_random_elements(results)

,pred_str,text
0,IT WAS IMPOSSIBLE TO RESIST AN EXISTENCE,XXDXXXDXXXXXXXXXXDXXDXXXXXXDXXDXXXXXXXXX
1,I HAVE TRAVELLED A GOOD DEAL AND SEEN MUCH GRANDER THINGS,XDXXXXDXXXXXXXXXDXDXXXXDXXXXDXXXDXXXXDXXXXDXXXXXXXDXXXXXX
2,BE AT PRUDENCE'S TO NIGHT AT EIGHT,XXDXXDXXXXXXXXAXDXXDXXXXXDXXDXXXXX
3,WELL SO LONG BOYS AND WE ALL WISH YOU SUCCESS,XXXXDXXDXXXXDXXXXDXXXDXXDXXXDXXXXDXXXDXXXXXXX
4,NO AT POINT DE JOUR WHERE WE HAD DINNER THE DUKE AND I,XXDXXDXXXXXDXXDXXXXDXXXXXDXXDXXXDXXXXXXDXXXDXXXXDXXXDX
5,IN THE COURSE OF THE DAY I RECEIVED THIS NOTE,XXDXXXDXXXXXXDXXDXXXDXXXDXDXXXXXXXXDXXXXDXXXX
6,AS THE FOUR CHUMS WENT AWAY JERRY CHUCKLED,XXDXXXDXXXXDXXXXXDXXXXDXXXXDXXXXXDXXXXXXXX
7,OH NO IT ISN'T SO BAD AS THAT HE WAS ASSURED,XXDXXDXXDXXXAXDXXDXXXDXXDXXXXDXXDXXXDXXXXXXX
8,ALTHOUGH THE LATTER HAD NOT BEEN A RESIDENT THERE MUCH MORE THAN THREE YEARS,XXXXXXXXDXXXDXXXXXXDXXXDXXXDXXXXDXDXXXXXXXXDXXXXXDXXXXDXXXXDXXXXDXXXXXDXXXXX
9,I HAPPENED TO DODGE A BALL FIRED FROM THE OTHER SIDE AND IT WENT THROUGH THE GLASS WHAT,XDXXXXXXXXDXXDXXXXXDXDXXXXDXXXXXDXXXXDXXXDXXXXXDXXXXDXXXDXXDXXXXDXXXXXXXDXXXDXXXXXDXXXX


In [113]:
with torch.no_grad():
  logits = mymodel(torch.tensor(vectorized_datasets["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[UNK] [UNK] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [UNK] | | [UNK] [UNK] [UNK] [PAD] [PAD] [UNK] | | [UNK] [PAD] [PAD] [PAD] [UNK] [UNK] [UNK] [UNK] [PAD] [PAD] [UNK] [PAD] [PAD] [UNK] [UNK] [UNK] [UNK] [PAD] [UNK] [UNK] [PAD] [PAD] [PAD] [UNK] [PAD] [PAD] [UNK] [PAD] [PAD] [UNK] [PAD] [PAD] [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] | [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] | [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [PAD] [UNK] [UNK] [UNK] [PAD] [PAD] [UNK] | | [UNK] [UNK] [UNK] [PAD] [PAD] [UNK] | [UNK] [UNK] [PAD] [PAD] [UNK] [PAD] [PAD] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [PAD] [UNK] [UNK] [UNK] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD] [UNK] [UNK] [UNK] [UNK] [UNK] [PAD] [PAD] [PAD] [UNK] [UNK] [PAD] [PAD]'

# Custom Training loop

In [30]:
from torch.utils.data.dataloader import DataLoader
batch_size = 8
train_dataloader = DataLoader(
    vectorized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    vectorized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [32]:
# Optimizer
learning_rate=5e-5
adam_beta1=0.9
adam_beta2=0.999
adam_epsilon=1e-8
optimizer = AdamW(
    list(model.parameters()),
    lr=learning_rate,
    betas=[adam_beta1, adam_beta2],
    eps=adam_epsilon,
)

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
gradient_accumulation_steps =1
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

In [34]:
num_update_steps_per_epoch

3459